In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from pandas import Series, DataFrame
from statsmodels.formula.api import ols
from statsmodels.api import MNLogit
import statsmodels.discrete.discrete_model as sm
from sklearn import preprocessing

# Data Cleaning

In [22]:
tau = 0
n = 500
q = 2
h = 5

In [23]:
data = pd.read_csv('STARD/S1_data_kNN_2.csv', index_col='Patient')


#data cleaning
data['L3_treatment'] = data['L3_treatment'].fillna(0)  #define no-treatment in the second stage as treatment 0
data['L3_End_QCTOT'] = data['L3_End_QCTOT'].fillna(data['L3_Int_QCTOT'])  #define the outcome for patients not treatedin the second stage

#drop the two rows where we treat someone with QCTOT == 5
data.L3_treatment[1246] = None
data.L3_treatment[3151] = None


data = data.dropna()

data.L2_Int_QCTOT = -data.L2_Int_QCTOT
data.L2_End_QCTOT = -data.L2_End_QCTOT
data.L3_Int_QCTOT = -data.L3_Int_QCTOT
data.L3_End_QCTOT = -data.L3_End_QCTOT
data['dropped'] = None
data['forced'] = None
data['p1'] = None
data['p0'] = None
data['p2'] = None
data['p'] = None
data['weight'] = None

data['A1_L2_Int_QCTOT'] = data.L2_treatment * data.L2_Int_QCTOT
data['A1_L1_slope'] = data.L2_treatment * data.L1_slope
data['A1_L2_preference'] = data.L2_treatment * data.L2_preference

/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [24]:
p1 = sm.Logit(data['L2_treatment']*(data.L2_treatment == 1), data[['L2_Int_QCTOT','L1_slope','L2_preference']]).fit().predict(data[['L2_Int_QCTOT','L1_slope','L2_preference']])
#data = data.join(p1.rename('p1'))
data.p1 = p1*(data.L2_treatment == 1) + (1-p1)*(data.L2_treatment == -1)
data

Optimization terminated successfully.
         Current function value: 0.435317
         Iterations 7


,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,dropped,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference
Patient,,,,,,,,,,,,,,,,,,,,,
3,-10.0,-0.500000,1,1,-8.0,-8.0,-0.222222,1.0,-14.277259,0,...,None,None,0.944244,None,None,None,None,-10.0,-0.500000,1
7,-15.0,-0.666667,-1,1,-20.0,-20.0,0.555556,-1.0,-35.578405,1,...,None,None,0.369651,None,None,None,None,-15.0,-0.666667,-1
10,-14.0,-0.666667,-1,-1,-8.0,-8.0,-0.666667,-1.0,-15.187805,0,...,None,None,0.648169,None,None,None,None,14.0,0.666667,1
16,-12.0,-1.000000,-1,-1,-10.0,-10.0,-0.166667,1.0,-18.090773,0,...,None,None,0.659646,None,None,None,None,12.0,1.000000,1
21,-15.0,-0.357143,1,1,-2.0,-2.0,-1.083333,-1.0,-4.813656,0,...,None,None,0.959741,None,None,None,None,-15.0,-0.357143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,-20.0,-0.500000,-1,-1,-9.0,-9.0,-1.833333,-1.0,-17.820692,0,...,None,None,0.549638,None,None,None,None,20.0,0.500000,1
4027,-10.0,-0.583333,1,1,-15.0,-15.0,0.357143,1.0,-25.820482,1,...,None,None,0.945595,None,None,None,None,-10.0,-0.583333,1
4029,-12.0,-0.500000,-1,-1,-3.0,-3.0,-0.750000,-1.0,-6.552238,0,...,None,None,0.693706,None,None,None,None,12.0,0.500000,1


In [25]:
#dropout probability
subdata = data[-data.L2_End_QCTOT > 5]
p0 = sm.Logit((subdata['L3_treatment']+1)*(subdata.L3_treatment == 0), subdata[['L2_Int_QCTOT','L1_slope','L2_preference','L2_treatment', 'L2_End_QCTOT', 'L2_slope', 'A1_L2_Int_QCTOT','A1_L1_slope','A1_L2_preference']]).fit()
p0 = p0.predict(subdata[['L2_Int_QCTOT','L1_slope','L2_preference','L2_treatment', 'L2_End_QCTOT', 'L2_slope', 'A1_L2_Int_QCTOT','A1_L1_slope','A1_L2_preference']])
#data = data.join(p0.rename('p0'))
#data.p0 = p0
p0

Optimization terminated successfully.
         Current function value: 0.629411
         Iterations 5


Patient
3       0.683170
7       0.323676
10      0.740680
16      0.638280
24      0.759657
          ...   
4014    0.342109
4023    0.875670
4027    0.420817
4030    0.727018
4033    0.603007
Length: 792, dtype: float64

In [26]:
data.p0 = p0*(data.L3_Indicator == 0) + (1-p0)*(data.L3_Indicator == 1)
data.p0[-data.L2_End_QCTOT < 6] = 1

/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
data

,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,dropped,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference
Patient,,,,,,,,,,,,,,,,,,,,,
3,-10.0,-0.500000,1,1,-8.0,-8.0,-0.222222,1.0,-14.277259,0,...,None,None,0.944244,0.683170,None,None,None,-10.0,-0.500000,1
7,-15.0,-0.666667,-1,1,-20.0,-20.0,0.555556,-1.0,-35.578405,1,...,None,None,0.369651,0.676324,None,None,None,-15.0,-0.666667,-1
10,-14.0,-0.666667,-1,-1,-8.0,-8.0,-0.666667,-1.0,-15.187805,0,...,None,None,0.648169,0.740680,None,None,None,14.0,0.666667,1
16,-12.0,-1.000000,-1,-1,-10.0,-10.0,-0.166667,1.0,-18.090773,0,...,None,None,0.659646,0.638280,None,None,None,12.0,1.000000,1
21,-15.0,-0.357143,1,1,-2.0,-2.0,-1.083333,-1.0,-4.813656,0,...,None,None,0.959741,1.000000,None,None,None,-15.0,-0.357143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,-20.0,-0.500000,-1,-1,-9.0,-9.0,-1.833333,-1.0,-17.820692,0,...,None,None,0.549638,0.875670,None,None,None,20.0,0.500000,1
4027,-10.0,-0.583333,1,1,-15.0,-15.0,0.357143,1.0,-25.820482,1,...,None,None,0.945595,0.579183,None,None,None,-10.0,-0.583333,1
4029,-12.0,-0.500000,-1,-1,-3.0,-3.0,-0.750000,-1.0,-6.552238,0,...,None,None,0.693706,1.000000,None,None,None,12.0,0.500000,1


In [28]:
# the prob of l3 = 1 given l2 = 1
subdata = data[(data.L3_Indicator == 1) & (data.L2_treatment == 1)]
p2 = sm.Logit((subdata['L3_treatment'])*(subdata.L3_treatment == 1), subdata[['L2_Int_QCTOT','L1_slope','L2_preference','L2_End_QCTOT', 'L2_slope', 'L3_preference']]).fit()
p2 = p2.predict(subdata[['L2_Int_QCTOT','L1_slope','L2_preference','L2_End_QCTOT', 'L2_slope', 'L3_preference']])
p2

Optimization terminated successfully.
         Current function value: 0.271596
         Iterations 7


Patient
7       0.055172
53      0.081545
66      0.064273
104     0.091350
177     0.081030
          ...   
3939    0.835632
3969    0.067938
3976    0.888978
4014    0.040399
4027    0.909865
Length: 189, dtype: float64

In [29]:
data.p2 = p2 * (data.L3_treatment == 1) * (data.L2_treatment == 1) + (1-p2) * (data.L3_treatment == -1) * (data.L2_treatment == 1)
data.p2 = data.p2.fillna(1)
data

,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,dropped,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference
Patient,,,,,,,,,,,,,,,,,,,,,
3,-10.0,-0.500000,1,1,-8.0,-8.0,-0.222222,1.0,-14.277259,0,...,None,None,0.944244,0.683170,1.000000,None,None,-10.0,-0.500000,1
7,-15.0,-0.666667,-1,1,-20.0,-20.0,0.555556,-1.0,-35.578405,1,...,None,None,0.369651,0.676324,0.944828,None,None,-15.0,-0.666667,-1
10,-14.0,-0.666667,-1,-1,-8.0,-8.0,-0.666667,-1.0,-15.187805,0,...,None,None,0.648169,0.740680,1.000000,None,None,14.0,0.666667,1
16,-12.0,-1.000000,-1,-1,-10.0,-10.0,-0.166667,1.0,-18.090773,0,...,None,None,0.659646,0.638280,1.000000,None,None,12.0,1.000000,1
21,-15.0,-0.357143,1,1,-2.0,-2.0,-1.083333,-1.0,-4.813656,0,...,None,None,0.959741,1.000000,1.000000,None,None,-15.0,-0.357143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,-20.0,-0.500000,-1,-1,-9.0,-9.0,-1.833333,-1.0,-17.820692,0,...,None,None,0.549638,0.875670,1.000000,None,None,20.0,0.500000,1
4027,-10.0,-0.583333,1,1,-15.0,-15.0,0.357143,1.0,-25.820482,1,...,None,None,0.945595,0.579183,0.909865,None,None,-10.0,-0.583333,1
4029,-12.0,-0.500000,-1,-1,-3.0,-3.0,-0.750000,-1.0,-6.552238,0,...,None,None,0.693706,1.000000,1.000000,None,None,12.0,0.500000,1


In [30]:
data['p'] = data.p1 * data.p0 * data.p2
data

,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,dropped,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference
Patient,,,,,,,,,,,,,,,,,,,,,
3,-10.0,-0.500000,1,1,-8.0,-8.0,-0.222222,1.0,-14.277259,0,...,None,None,0.944244,0.683170,1.000000,0.645079,None,-10.0,-0.500000,1
7,-15.0,-0.666667,-1,1,-20.0,-20.0,0.555556,-1.0,-35.578405,1,...,None,None,0.369651,0.676324,0.944828,0.236211,None,-15.0,-0.666667,-1
10,-14.0,-0.666667,-1,-1,-8.0,-8.0,-0.666667,-1.0,-15.187805,0,...,None,None,0.648169,0.740680,1.000000,0.480086,None,14.0,0.666667,1
16,-12.0,-1.000000,-1,-1,-10.0,-10.0,-0.166667,1.0,-18.090773,0,...,None,None,0.659646,0.638280,1.000000,0.421039,None,12.0,1.000000,1
21,-15.0,-0.357143,1,1,-2.0,-2.0,-1.083333,-1.0,-4.813656,0,...,None,None,0.959741,1.000000,1.000000,0.959741,None,-15.0,-0.357143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,-20.0,-0.500000,-1,-1,-9.0,-9.0,-1.833333,-1.0,-17.820692,0,...,None,None,0.549638,0.875670,1.000000,0.481301,None,20.0,0.500000,1
4027,-10.0,-0.583333,1,1,-15.0,-15.0,0.357143,1.0,-25.820482,1,...,None,None,0.945595,0.579183,0.909865,0.498308,None,-10.0,-0.583333,1
4029,-12.0,-0.500000,-1,-1,-3.0,-3.0,-0.750000,-1.0,-6.552238,0,...,None,None,0.693706,1.000000,1.000000,0.693706,None,12.0,0.500000,1


In [31]:
#tau = 0.1
data.p = data.p.clip(tau,1)
data.weight = (1/data.p).apply(np.floor)

In [32]:
data

,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,dropped,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference
Patient,,,,,,,,,,,,,,,,,,,,,
3,-10.0,-0.500000,1,1,-8.0,-8.0,-0.222222,1.0,-14.277259,0,...,None,None,0.944244,0.683170,1.000000,0.645079,1.0,-10.0,-0.500000,1
7,-15.0,-0.666667,-1,1,-20.0,-20.0,0.555556,-1.0,-35.578405,1,...,None,None,0.369651,0.676324,0.944828,0.236211,4.0,-15.0,-0.666667,-1
10,-14.0,-0.666667,-1,-1,-8.0,-8.0,-0.666667,-1.0,-15.187805,0,...,None,None,0.648169,0.740680,1.000000,0.480086,2.0,14.0,0.666667,1
16,-12.0,-1.000000,-1,-1,-10.0,-10.0,-0.166667,1.0,-18.090773,0,...,None,None,0.659646,0.638280,1.000000,0.421039,2.0,12.0,1.000000,1
21,-15.0,-0.357143,1,1,-2.0,-2.0,-1.083333,-1.0,-4.813656,0,...,None,None,0.959741,1.000000,1.000000,0.959741,1.0,-15.0,-0.357143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,-20.0,-0.500000,-1,-1,-9.0,-9.0,-1.833333,-1.0,-17.820692,0,...,None,None,0.549638,0.875670,1.000000,0.481301,2.0,20.0,0.500000,1
4027,-10.0,-0.583333,1,1,-15.0,-15.0,0.357143,1.0,-25.820482,1,...,None,None,0.945595,0.579183,0.909865,0.498308,2.0,-10.0,-0.583333,1
4029,-12.0,-0.500000,-1,-1,-3.0,-3.0,-0.750000,-1.0,-6.552238,0,...,None,None,0.693706,1.000000,1.000000,0.693706,1.0,12.0,0.500000,1


In [33]:
data = data.sample(n = int(data.weight.sum()), replace=True, weights='weight', random_state=1)
data.index = np.arange(len(data))
data

,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,dropped,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference
0,-17.0,0.166667,-1,-1,-13.0,-13.0,-0.285714,-1.0,-24.730100,1,...,None,None,0.654478,0.431432,1.000000,0.282363,3.0,17.0,-0.166667,1
1,-13.0,-0.250000,1,1,-4.0,-4.0,-1.500000,1.0,-8.682913,0,...,None,None,0.951817,1.000000,1.000000,0.951817,1.0,-13.0,-0.250000,1
2,-10.0,-0.500000,1,1,-8.0,-8.0,-0.222222,1.0,-14.277259,0,...,None,None,0.944244,0.683170,1.000000,0.645079,1.0,-10.0,-0.500000,1
3,-14.0,-0.250000,-1,1,-12.0,-12.0,-0.222222,-1.0,-22.522760,1,...,None,None,0.322824,0.427960,0.945934,0.130686,7.0,-14.0,-0.250000,-1
4,-23.0,0.555556,-1,1,-11.0,-11.0,-3.000000,-1.0,-21.570821,0,...,None,None,0.426480,0.660799,1.000000,0.281817,3.0,-23.0,0.555556,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,-8.0,-0.333333,-1,1,-3.0,-3.0,-0.416667,1.0,-5.772945,0,...,None,None,0.235307,1.000000,1.000000,0.235307,4.0,-8.0,-0.333333,-1
4196,-10.0,-0.500000,-1,-1,-15.0,-15.0,2.500000,-1.0,-24.933026,1,...,None,None,0.725533,0.708745,1.000000,0.514219,1.0,10.0,0.500000,1
4197,-8.0,-2.166667,-1,1,-12.0,-12.0,0.285714,-1.0,-21.922751,1,...,None,None,0.352661,0.342223,0.096395,0.011634,85.0,-8.0,-2.166667,-1
4198,-9.0,-0.428571,-1,1,-8.0,-8.0,-0.071429,-1.0,-14.099132,0,...,None,None,0.255095,0.656473,1.000000,0.167463,5.0,-9.0,-0.428571,-1


In [34]:
data.mean()

L2_Int_QCTOT       -13.564762
L1_slope            -0.404690
L2_preference       -0.215238
L2_treatment         0.226190
L2_End_QCTOT       -10.253095
L3_Int_QCTOT       -10.431905
L2_slope            -0.346504
L3_preference       -0.241905
L2_Pseudo          -18.968622
L3_Indicator         0.441190
L3_treatment        -0.111190
L3_End_QCTOT        -9.675000
dropped                   NaN
forced                    NaN
p1                   0.510098
p0                   0.593460
p2                   0.832794
p                    0.251936
weight              20.158571
A1_L2_Int_QCTOT     -2.388571
A1_L1_slope         -0.129900
A1_L2_preference    -0.055714
dtype: float64

In [35]:
(data.L2_End_QCTOT/2 + data.L3_End_QCTOT/2).mean()

-9.96404761904762

In [36]:
#n = 500
(data.L2_End_QCTOT.head(n)/2 + data.L3_End_QCTOT.head(n)/2).mean()

-9.912

# DTR Bandit

In [37]:
#parameters
#q = 1
#h = 10
K = 2
d = 3
T = 4000

#forced pull index
forced_index_minus1 = np.zeros(T, dtype = bool)
forced_index_minus1[0:d] = True
forced_index_1 = np.zeros(T, dtype = bool)
forced_index_1[d:(2*d)] = True


for i in range(16):
    for j in range(q):
        if (2**i - 1) * K * q + j > 2*d - 1 and (2**i - 1) * K * q + j < T:
            forced_index_minus1[(2**i - 1) * K * q + j] = True
        if (2**i - 1) * K * q + q + j> 2*d - 1 and (2**i - 1) * K * q + q + j < T:
            forced_index_1[(2**i - 1) * K * q + q + j] = True

In [38]:
n_total = 0 #total number of points

#the first period beta's are 0
for i in data.index:
    
    # compute our chosen arm
    if forced_index_1[n_total]:
        Forced = True
        A_1 = 1
        A_2 = 1
        
    elif forced_index_minus1[n_total]:
        Forced = True
        A_1 = -1
        A_2 = -1
        
    else:
        Forced = False      
        X_i = data[data.index == i]
        
        #first period
        #estimate Q Tilde 1
        #may also try deleting the "force==True" condition
        data_forced_minus1 = data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == -1)]
        data_forced_1 = data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1)]
        temp_minus1 = pd.concat([- B1_tilde_minus1.predict(data_forced_minus1), - B2_tilde_minus1.predict(data_forced_minus1), - B3_tilde_minus1.predict(data_forced_minus1)], axis = 1)
        temp_minus1 = temp_minus1 + pd.concat([B1_tilde_minus1.predict(X_i), B2_tilde_minus1.predict(X_i), B3_tilde_minus1.predict(X_i)], axis = 1).iloc[0]
        temp_minus1 = temp_minus1.rename(columns={0: "L3_Int_QCTOT", 1: "L2_slope", 2:"L3_preference"})
        temp_minus1 = data_forced_minus1.add(temp_minus1, fill_value = 0) 
        temp_1 = pd.concat([- B1_tilde_1.predict(data_forced_1), - B2_tilde_1.predict(data_forced_1), - B3_tilde_1.predict(data_forced_1)], axis = 1)
        temp_1 = temp_1 + pd.concat([B1_tilde_1.predict(X_i), B2_tilde_1.predict(X_i), B3_tilde_1.predict(X_i)], axis = 1).iloc[0]
        temp_1 = temp_1.rename(columns={0: "L3_Int_QCTOT", 1: "L2_slope", 2:"L3_preference"})
        temp_1 = data_forced_1.add(temp_1, fill_value = 0)
        Q_tilde_minus1 = beta_tilde_minus11.predict(X_i).iloc[0] + np.vstack((beta_tilde_minus12.predict(temp_minus1), beta_tilde_12.predict(temp_minus1), beta_tilde_minus11.predict(X_i).iloc[0]*np.ones(data_forced_minus1.shape[0]))).max(axis = 0).mean()
        Q_tilde_1 = beta_tilde_11.predict(X_i).iloc[0] + np.vstack((beta_tilde_minus12.predict(temp_1), beta_tilde_12.predict(temp_1), beta_tilde_11.predict(X_i).iloc[0]*np.ones(data_forced_1.shape[0]))).max(axis = 0).mean()
        
        
        if Q_tilde_minus1 > Q_tilde_1 + h:  
            A_1 = -1
        elif Q_tilde_1 > Q_tilde_minus1 + h:
            A_1 = 1
        else:            
            #estimate Q hat 1
            data_minus1 = data[(data.dropped == False)  & (data.L2_treatment == -1)]
            data_1 = data[(data.dropped == False)  & (data.L2_treatment == 1)]
            temp_minus1 = pd.concat([- B1_hat_minus1.predict(data_minus1), - B2_hat_minus1.predict(data_minus1), - B3_hat_minus1.predict(data_minus1)], axis = 1)
            temp_minus1 = temp_minus1 + pd.concat([B1_hat_minus1.predict(X_i), B2_hat_minus1.predict(X_i), B3_hat_minus1.predict(X_i)], axis = 1).iloc[0]       
            temp_minus1 = temp_minus1.rename(columns={0: "L3_Int_QCTOT", 1: "L2_slope", 2:"L3_preference"})
            temp_minus1 = data_minus1.add(temp_minus1, fill_value = 0) 
            temp_1 = pd.concat([- B1_hat_1.predict(data_1), - B2_hat_1.predict(data_1), - B3_hat_1.predict(data_1)], axis = 1)
            temp_1 = temp_1 + pd.concat([B1_hat_1.predict(X_i), B2_hat_1.predict(X_i), B3_hat_1.predict(X_i)], axis = 1).iloc[0]
            temp_1 = temp_1.rename(columns={0: "L3_Int_QCTOT", 1: "L2_slope", 2:"L3_preference"})
            temp_1 = data_1.add(temp_1, fill_value = 0)
            Q_hat_minus1 = beta_hat_minus11.predict(X_i).iloc[0] + np.vstack((beta_hat_minus12.predict(temp_minus1), beta_hat_12.predict(temp_minus1), beta_hat_minus11.predict(X_i).iloc[0]*np.ones(data_minus1.shape[0]))).max(axis = 0).mean()
            Q_hat_1 = beta_hat_11.predict(X_i).iloc[0] + np.vstack((beta_hat_minus12.predict(temp_1), beta_hat_12.predict(temp_1), beta_hat_11.predict(X_i).iloc[0]*np.ones(data_1.shape[0]))).max(axis = 0).mean()
            
                    
            if Q_hat_minus1 > Q_hat_1:
                A_1 = -1
            else:
                A_1 = 1
               
        
        
        #second period (three possible treatments) 
        if - data.L2_End_QCTOT[i] < 6:
            A_2 = 0
        else:
            value_tilde = np.array([beta_tilde_minus12.predict(X_i).iloc[0], data.L3_Int_QCTOT[i], beta_tilde_12.predict(X_i).iloc[0]])
            value_hat = np.array([beta_hat_minus12.predict(X_i).iloc[0], data.L3_Int_QCTOT[i], beta_hat_12.predict(X_i).iloc[0]])
            value_max = value_hat[value_tilde > value_tilde.max()- h/2].max()
        
            if value_max == beta_hat_minus12.predict(X_i).iloc[0] and (value_tilde > value_tilde.max() - h/2)[0]:
                A_2 = -1
            elif value_max == data.L3_Int_QCTOT[i] and (value_tilde > value_tilde.max() - h/2)[1]:
                A_2 = 0
            else:
                A_2 = 1
        
    
    
    # if disagrees, we drop the point; else, we update estimators
    if A_1 == data.L2_treatment[i] and A_2 == data.L3_treatment[i]:
        print(i)
        data.dropped[i] = False
        data.forced[i] = Forced
        n_total = n_total + 1
        
        #estimate tilde parameters if forced = True
        if data.forced[i] and n_total >= 2*d:
            B1_tilde_minus1 = ols('L3_Int_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == -1)]).fit() 
            B2_tilde_minus1 = ols('L2_slope ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == -1)]).fit() 
            B3_tilde_minus1 = ols('L3_preference ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == -1)]).fit() 
            
            B1_tilde_1 = ols('L3_Int_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1)]).fit() 
            B2_tilde_1 = ols('L2_slope ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1)]).fit() 
            B3_tilde_1 = ols('L3_preference ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1)]).fit() 
            
            beta_tilde_minus11 = ols('L2_End_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == -1)]).fit() 
            beta_tilde_11 = ols('L2_End_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1)]).fit() 
            beta_tilde_minus12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L3_treatment == -1)]).fit() 
            beta_tilde_12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L3_treatment == 1)]).fit() 
        
        #estimate hat parameters
        if n_total >= 2*d:
            B1_hat_minus1 = ols('L3_Int_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == -1)]).fit() 
            B2_hat_minus1 = ols('L2_slope ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == -1)]).fit() 
            B3_hat_minus1 = ols('L3_preference ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == -1)]).fit() 
            
            B1_hat_1 = ols('L3_Int_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == 1)]).fit() 
            B2_hat_1 = ols('L2_slope ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == 1)]).fit() 
            B3_hat_1 = ols('L3_preference ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == 1)]).fit() 
            
            beta_hat_minus11 = ols('L2_End_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == -1)]).fit() 
            beta_hat_11 = ols('L2_End_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == 1)]).fit() 
            beta_hat_minus12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False)  & (data.L3_treatment == -1)]).fit() 
            beta_hat_12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False)  & (data.L3_treatment == 1)]).fit() 
        
        
    else:
        data.dropped[i] = True 
        
    if n_total >= n:
        break

0
9
13
30
44
45


/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


46
60
61
63
74
80
101
107
117
136
137
151
152
154
157
159
160
162
167
169
173
177
178
183
185
189
191
192
198
201
204
205
210
214
218
226
231
232
233
234
235
237
238
240
246
252
258
259
260
263
264
266
268
272
284
290
291
309
316
321
324
327
328
329
330
346
356
358
359
363
364
367
368
370
371
372
373
381
385
386
388
391
394
396
397
399
400
401
412
433
434
435
440
446
448
457
464
468
475
487
489
490
492
499
500
501
503
505
508
510
512
516
517
518
521
524
525
527
531
538
543
547
561
577
585
590
596
598
600
602
604
606
610
616
617
618
621
623
633
634
635
636
640
656
657
662
676
677
680
683
686
687
692
694
698
707
709
720
728
729
733
734
736
745
749
753
755
768
774
781
786
787
788
789
790
794
797
800
806
810
813
816
817
819
824
826
827
831
845
847
854
856
857
860
861
862
868
869
872
877
878
880
883
884
892
893
900
904
907
910
922
928
932
933
942
950
957
960
961
962
966
971
977
985
995
1002
1004
1005
1021
1028
1032
1036
1041
1046
1049
1053
1054
1061
1064
1070
1075
1096
1097
1098
1101
1102
1

In [39]:
Reward = (data.L2_End_QCTOT[data.dropped == False] + data.L3_End_QCTOT[data.dropped == False])/2     
Reward.mean()

-7.999

In [40]:
temp = data[data.dropped == False]
temp.tail(50)

,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,dropped,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference
2103,-14.0,-1.166667,-1,-1,-12.0,-12.0,-0.142857,1.0,-21.556250,1,...,False,False,0.611878,0.315690,1.000000,0.193164,5.0,14.0,1.166667,1
2107,-8.0,-0.428571,1,-1,-2.0,-2.0,-0.500000,1.0,-4.510120,0,...,False,False,0.065831,1.000000,1.000000,0.065831,15.0,8.0,0.428571,-1
2113,-12.0,-0.333333,-1,1,-4.0,-4.0,-2.000000,-1.0,-8.990060,0,...,False,False,0.295372,1.000000,1.000000,0.295372,3.0,-12.0,-0.333333,-1
2122,-10.0,-0.642857,1,-1,-5.0,-5.0,-0.208333,1.0,-9.405987,0,...,False,False,0.053459,1.000000,1.000000,0.053459,18.0,10.0,0.642857,-1
2124,-15.0,0.750000,-1,1,-11.0,-11.0,-0.285714,-1.0,-20.850960,1,...,False,False,0.273863,0.471918,0.958342,0.123857,8.0,-15.0,0.750000,-1
2129,-16.0,0.000000,1,-1,-3.0,-3.0,-0.928571,-1.0,-6.763179,0,...,False,False,0.041593,1.000000,1.000000,0.041593,24.0,16.0,-0.000000,-1
2130,-19.0,0.000000,-1,-1,-20.0,-20.0,0.111111,-1.0,-36.488950,1,...,False,False,0.606418,0.579830,1.000000,0.351619,2.0,19.0,-0.000000,1
2135,-15.0,-0.166667,-1,1,-16.0,-16.0,0.166667,-1.0,-29.050582,1,...,False,False,0.334144,0.575986,0.949742,0.182790,5.0,-15.0,-0.166667,-1
2138,-15.0,0.750000,-1,1,-11.0,-11.0,-0.285714,-1.0,-20.850960,1,...,False,False,0.273863,0.471918,0.958342,0.123857,8.0,-15.0,0.750000,-1
2139,-19.0,0.166667,-1,1,-16.0,-16.0,-0.750000,-1.0,-30.133412,1,...,False,False,0.381259,0.546247,0.955816,0.199059,5.0,-19.0,0.166667,-1


In [41]:
(temp.L2_End_QCTOT/2 -temp.L2_Int_QCTOT + temp.L3_End_QCTOT/2).mean()

5.701

# Greedy to Q learning; 3 actions in the second step

In [ ]:
data['dropped'] = None

In [ ]:
n_total = 0 #total number of points

#the first period beta's are 0
for i in data.index:
    
    # compute our chosen arm
    if n_total < d:
        A_1 = -1
        A_2 = -1
        
    elif n_total < 2*d:
        A_1 = 1
        A_2 = 1
        
    else:     
        X_i = data[data.index == i]
        
        #first period
        #estimate Q hat 1
        data_minus1 = data[(data.dropped == False)  & (data.L2_treatment == -1)]
        data_1 = data[(data.dropped == False)  & (data.L2_treatment == 1)]
        temp_minus1 = pd.concat([- B1_hat_minus1.predict(data_minus1), - B2_hat_minus1.predict(data_minus1), - B3_hat_minus1.predict(data_minus1)], axis = 1)
        temp_minus1 = temp_minus1 + pd.concat([B1_hat_minus1.predict(X_i), B2_hat_minus1.predict(X_i), B3_hat_minus1.predict(X_i)], axis = 1).iloc[0]       
        temp_minus1 = temp_minus1.rename(columns={0: "L3_Int_QCTOT", 1: "L2_slope", 2:"L3_preference"})
        temp_minus1 = data_minus1.add(temp_minus1, fill_value = 0) 
        temp_1 = pd.concat([- B1_hat_1.predict(data_1), - B2_hat_1.predict(data_1), - B3_hat_1.predict(data_1)], axis = 1)
        temp_1 = temp_1 + pd.concat([B1_hat_1.predict(X_i), B2_hat_1.predict(X_i), B3_hat_1.predict(X_i)], axis = 1).iloc[0]
        temp_1 = temp_1.rename(columns={0: "L3_Int_QCTOT", 1: "L2_slope", 2:"L3_preference"})
        temp_1 = data_1.add(temp_1, fill_value = 0)
        Q_hat_minus1 = beta_hat_minus11.predict(X_i).iloc[0] + np.vstack((beta_hat_minus12.predict(temp_minus1), beta_hat_12.predict(temp_minus1), beta_hat_minus11.predict(X_i).iloc[0]*np.ones(data_minus1.shape[0]))).max(axis = 0).mean()
        Q_hat_1 = beta_hat_11.predict(X_i).iloc[0] + np.vstack((beta_hat_minus12.predict(temp_1), beta_hat_12.predict(temp_1), beta_hat_11.predict(X_i).iloc[0]*np.ones(data_1.shape[0]))).max(axis = 0).mean()
                
                    
        if Q_hat_minus1 > Q_hat_1:
            A_1 = -1
        else:
            A_1 = 1
               
        
        
        #second period (three possible treatments)
        if -data.L2_End_QCTOT[i] < 6:
            A_2 = 0
        else:
            value_hat = np.array([beta_hat_minus12.predict(X_i).iloc[0], data.L3_Int_QCTOT[i], beta_hat_12.predict(X_i).iloc[0]])
            value_max = value_hat.max()
        
            if value_max == beta_hat_minus12.predict(X_i).iloc[0]:
                A_2 = -1
            elif value_max == data.L3_Int_QCTOT[i]:
                A_2 = 0
            else:
                A_2 = 1
        
    
    
    # if disagrees, we drop the point; else, we update estimators
    if A_1 == data.L2_treatment[i] and A_2 == data.L3_treatment[i]:
        print(i)
        data.dropped[i] = False
        n_total = n_total + 1
        
        #estimate hat parameters
        if n_total >= 2*d:
            B1_hat_minus1 = ols('L3_Int_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == -1)]).fit() 
            B2_hat_minus1 = ols('L2_slope ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == -1)]).fit() 
            B3_hat_minus1 = ols('L3_preference ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == -1)]).fit() 
            
            B1_hat_1 = ols('L3_Int_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == 1)]).fit() 
            B2_hat_1 = ols('L2_slope ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == 1)]).fit() 
            B3_hat_1 = ols('L3_preference ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == 1)]).fit() 
            
            beta_hat_minus11 = ols('L2_End_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == -1)]).fit() 
            beta_hat_11 = ols('L2_End_QCTOT ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False)  & (data.L2_treatment == 1)]).fit() 
            beta_hat_minus12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False)  & (data.L3_treatment == -1)]).fit() 
            beta_hat_12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False)  & (data.L3_treatment == 1)]).fit() 
        
        
    else:
        data.dropped[i] = True   
        
    if n_total >= n:
        break

In [ ]:
Reward = (data.L2_End_QCTOT[data.dropped == False] + data.L3_End_QCTOT[data.dropped == False])/2     
Reward.mean()

In [ ]:
temp = data[data.dropped == False]
temp.tail(50)

In [ ]:
(temp.L2_End_QCTOT/2 -temp.L2_Int_QCTOT + temp.L3_End_QCTOT/2).mean()

# Six arms linear 2

In [28]:
data['dropped'] = None
data['forced'] = None

In [29]:
n_total = 0 #total number of points

#the first period beta's are 0
for i in data.index:
    
    # compute our chosen arm
    if forced_index_minus1minus1[n_total]:
        Forced = True
        A_1 = -1
        A_2 = -1
        
    elif forced_index_minus10[n_total]:
        Forced = True
        A_1 = -1
        A_2 = 0
        
    elif forced_index_1minus1[n_total]:
        Forced = True
        A_1 = 1
        A_2 = -1
    
    elif forced_index_10[n_total]:
        Forced = True
        A_1 = 1
        A_2 = 0
        
    elif forced_index_11[n_total]:
        Forced = True
        A_1 = 1
        A_2 = 1
        
    else:
        Forced = False      
        X_i = data[data.index == i]
        
        value_tilde = np.array([beta_tilde_minus1minus1.predict(X_i).iloc[0],beta_tilde_minus10.predict(X_i).iloc[0],beta_tilde_1minus1.predict(X_i).iloc[0],beta_tilde_10.predict(X_i).iloc[0],beta_tilde_11.predict(X_i).iloc[0]])
        value_hat = np.array([beta_hat_minus1minus1.predict(X_i).iloc[0],beta_hat_minus10.predict(X_i).iloc[0],beta_hat_1minus1.predict(X_i).iloc[0],beta_hat_10.predict(X_i).iloc[0],beta_hat_11.predict(X_i).iloc[0]])
        value_max = value_hat[value_tilde > value_tilde.max() - h/2].max()
        
        if value_max == beta_hat_minus1minus1.predict(X_i).iloc[0] and (value_tilde > value_tilde.max() - h/2)[0]:
            A_1 = -1
            A_2 = -1
        elif value_max == beta_hat_minus10.predict(X_i).iloc[0] and (value_tilde > value_tilde.max() - h/2)[1]:
            A_1 = -1
            A_2 = 0
        elif value_max == beta_hat_1minus1.predict(X_i).iloc[0] and (value_tilde > value_tilde.max() - h/2)[2]:
            A_1 = 1
            A_2 = -1
        elif value_max == beta_hat_10.predict(X_i).iloc[0] and (value_tilde > value_tilde.max() - h/2)[3]:
            A_1 = 1
            A_2 = 0
        else:
            A_1 = 1
            A_2 = 1
            
        if -data.L2_End_QCTOT[i] < 6:
            A_2 = 0
        
    
    
    # if disagrees, we drop the point; else, we update estimators
    if A_1 == data.L2_treatment[i] and A_2 == data.L3_treatment[i]:
        print(i)
        data.dropped[i] = False
        data.forced[i] = Forced
        n_total = n_total + 1
        
        #estimate tilde parameters if forced = True
        if data.forced[i] and n_total >= K*d:
            beta_tilde_minus1minus1 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == -1) & (data.L3_treatment == -1)]).fit() 
            beta_tilde_minus10 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == -1) & (data.L3_treatment == 0)]).fit() 
            beta_tilde_1minus1 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1) & (data.L3_treatment == -1)]).fit() 
            beta_tilde_10 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1) & (data.L3_treatment == 0)]).fit() 
            beta_tilde_11 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1) & (data.L3_treatment == 1)]).fit() 
            
        #estimate hat parameters
        if n_total >= K*d:
            beta_hat_minus1minus1 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.L2_treatment == -1) & (data.L3_treatment == -1)]).fit() 
            beta_hat_minus10 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) &  (data.L2_treatment == -1) & (data.L3_treatment == 0)]).fit() 
            beta_hat_1minus1 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) &  (data.L2_treatment == 1) & (data.L3_treatment == -1)]).fit() 
            beta_hat_10 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) &  (data.L2_treatment == 1) & (data.L3_treatment == 0)]).fit() 
            beta_hat_11 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) &  (data.L2_treatment == 1) & (data.L3_treatment == 1)]).fit() 
              
    else:
        data.dropped[i] = True 
        
    if n_total >= n:
        break

0
9
13
14
21
24
41
42
43
48
51
57
60
70
73


/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


77
78
82
90
92
93
95
100
105
106
109
111
113
115
116
120
125
132
134
140
143
144
146
147
170
173
175
177
182
184
190
192
193
198
201
214
218
219
220
223
226
227
228
231
233
234
236
241
242
246
247
255
259
262
263
264
284
286
288
293
297
298
300
309
310
320
321
327
329
335
338
342
346
349
350
356
358
359
360
369
371
374
377
381
385
388
393
396
402
405
408
414
418
423
426
427
433
440
443
446
449
464
466
475
481
483
485
490
497
508
510
512
513
518
525
534
541
542
546
552
555
563
571
572
575
582
590
597
598
601
608
609
627
634
635
662
669
673
676
691
694
700
706
710
713
725
728
729
734
738
745
757
765
766
768
774
777
783
787
789
797
808
819
827
830
831
832
843
844
854
855
857
861
869
874
877
878
880
884
893
907
910
911
920
923
927
932
938
950
956
960
962
964
966
977
985
987
1002
1004
1010
1012
1019
1021
1022
1025
1026
1028
1032
1033
1036
1039
1042
1045
1055
1061
1064
1066
1078
1079
1080
1081
1082
1096
1105
1107
1108
1115
1116
1120
1123
1128
1138
1139
1145
1147
1148
1149
1151
1154
1158
1165

In [30]:
Reward = (data.L2_End_QCTOT[data.dropped == False] + data.L3_End_QCTOT[data.dropped == False])/2     
Reward.mean()

-8.437

In [31]:
temp = data[data.dropped == False]
temp.tail(50)

,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference,reward
2052,-14.0,0.222222,-1,-1,-7.0,-7.0,-0.500000,-1.0,-13.244126,0,...,False,0.708468,0.643439,1.0,0.455856,2.0,14.0,-0.222222,1,-7.0
2070,-11.0,-0.833333,1,-1,-1.0,-1.0,-0.714286,1.0,-3.016448,0,...,False,0.046952,1.000000,1.0,0.046952,21.0,11.0,0.833333,-1,-1.0
2072,-16.0,0.222222,1,-1,-21.0,-21.0,0.192308,-1.0,-38.139837,1,...,False,0.044443,0.821592,1.0,0.036514,27.0,16.0,-0.222222,-1,-19.0
2075,-14.0,0.166667,-1,-1,-9.0,-9.0,-2.500000,1.0,-18.230221,0,...,False,0.704880,0.549409,1.0,0.387267,2.0,14.0,-0.166667,1,-9.0
2076,-12.0,-1.000000,-1,1,-2.0,-2.0,-0.833333,1.0,-4.518339,0,...,False,0.340354,1.000000,1.0,0.340354,2.0,-12.0,-1.000000,-1,-2.0
2083,-16.0,0.222222,1,-1,-21.0,-21.0,0.192308,-1.0,-38.139837,1,...,False,0.044443,0.821592,1.0,0.036514,27.0,16.0,-0.222222,-1,-19.0
2085,-11.0,-0.333333,1,-1,-9.0,-9.0,-0.111111,-1.0,-17.536642,1,...,False,0.054436,0.552393,1.0,0.030070,33.0,11.0,0.333333,-1,-11.0
2092,-16.0,0.222222,1,-1,-21.0,-21.0,0.192308,-1.0,-38.139837,1,...,False,0.044443,0.821592,1.0,0.036514,27.0,16.0,-0.222222,-1,-19.0
2096,-11.0,0.000000,-1,-1,-8.0,-8.0,-0.214286,-1.0,-14.653422,0,...,False,0.740886,0.493934,1.0,0.365949,2.0,11.0,-0.000000,1,-8.0
2104,-11.0,-0.555556,-1,1,-11.0,-11.0,0.000000,1.0,-19.255159,0,...,False,0.293700,0.589871,1.0,0.173245,5.0,-11.0,-0.555556,-1,-11.0


In [32]:
(temp.L2_End_QCTOT/2 -temp.L2_Int_QCTOT + temp.L3_End_QCTOT/2).mean()

5.197

# Six arms + three arm

In [33]:
data['dropped'] = None
data['forced'] = None

In [34]:
n_total = 0 #total number of points

#the first period beta's are 0
for i in data.index:
    
    # compute our chosen arm
    if forced_index_minus1minus1[n_total]:
        Forced = True
        A_1 = -1
        A_2 = -1
        
    elif forced_index_minus10[n_total]:
        Forced = True
        A_1 = -1
        A_2 = 0
        
    elif forced_index_1minus1[n_total]:
        Forced = True
        A_1 = 1
        A_2 = -1
    
    elif forced_index_10[n_total]:
        Forced = True
        A_1 = 1
        A_2 = 0
        
    elif forced_index_11[n_total]:
        Forced = True
        A_1 = 1
        A_2 = 1
        
    else:
        Forced = False      
        X_i = data[data.index == i]
        
        
        #first period
        value_tilde = np.array([beta_tilde_minus1minus1.predict(X_i).iloc[0],beta_tilde_minus10.predict(X_i).iloc[0],beta_tilde_1minus1.predict(X_i).iloc[0],beta_tilde_10.predict(X_i).iloc[0],beta_tilde_11.predict(X_i).iloc[0]])
        value_hat = np.array([beta_hat_minus1minus1.predict(X_i).iloc[0],beta_hat_minus10.predict(X_i).iloc[0],beta_hat_1minus1.predict(X_i).iloc[0],beta_hat_10.predict(X_i).iloc[0],beta_hat_11.predict(X_i).iloc[0]])
        value_max = value_hat[value_tilde > value_tilde.max() - h/2].max()
        
        if value_max == beta_hat_minus1minus1.predict(X_i).iloc[0] and (value_tilde > value_tilde.max() - h/2)[0]:
            A_1 = -1
        elif value_max == beta_hat_minus10.predict(X_i).iloc[0] and (value_tilde > value_tilde.max() - h/2)[1]:
            A_1 = -1
        else:
            A_1 = 1
            
        
        #second period
        if -data.L2_End_QCTOT[i] < 6:
            A_2 = 0
        else:
            value_tilde = np.array([beta_tilde_minus12.predict(X_i).iloc[0], data.L3_Int_QCTOT[i], beta_tilde_12.predict(X_i).iloc[0]])
            value_hat = np.array([beta_hat_minus12.predict(X_i).iloc[0], data.L3_Int_QCTOT[i], beta_hat_12.predict(X_i).iloc[0]])
            value_max = value_hat[value_tilde > value_tilde.max()- h/2].max()
        
            if value_max == beta_hat_minus12.predict(X_i).iloc[0] and (value_tilde > value_tilde.max() - h/2)[0]:
                A_2 = -1
            elif value_max == data.L3_Int_QCTOT[i] and (value_tilde > value_tilde.max() - h/2)[1]:
                A_2 = 0
            else:
                A_2 = 1
        
    
    
    # if disagrees, we drop the point; else, we update estimators
    if A_1 == data.L2_treatment[i] and A_2 == data.L3_treatment[i]:
        print(i)
        data.dropped[i] = False
        data.forced[i] = Forced
        n_total = n_total + 1
        
        #estimate tilde parameters if forced = True
        if data.forced[i] and n_total >= K*d:
            beta_tilde_minus1minus1 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == -1) & (data.L3_treatment == -1)]).fit() 
            beta_tilde_minus10 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == -1) & (data.L3_treatment == 0)]).fit() 
            beta_tilde_1minus1 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1) & (data.L3_treatment == -1)]).fit() 
            beta_tilde_10 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1) & (data.L3_treatment == 0)]).fit() 
            beta_tilde_11 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L2_treatment == 1) & (data.L3_treatment == 1)]).fit() 
            beta_tilde_minus12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L3_treatment == -1)]).fit() 
            beta_tilde_12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False) & (data.forced == True) & (data.L3_treatment == 1)]).fit() 
            
        #estimate hat parameters
        if n_total >= K*d:
            beta_hat_minus1minus1 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) & (data.L2_treatment == -1) & (data.L3_treatment == -1)]).fit() 
            beta_hat_minus10 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) &  (data.L2_treatment == -1) & (data.L3_treatment == 0)]).fit() 
            beta_hat_1minus1 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) &  (data.L2_treatment == 1) & (data.L3_treatment == -1)]).fit() 
            beta_hat_10 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) &  (data.L2_treatment == 1) & (data.L3_treatment == 0)]).fit() 
            beta_hat_11 = ols('reward ~ L2_Int_QCTOT + L1_slope + L2_preference', data= data[(data.dropped == False) &  (data.L2_treatment == 1) & (data.L3_treatment == 1)]).fit() 
            beta_hat_minus12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False) &  (data.L3_treatment == -1)]).fit() 
            beta_hat_12 = ols('L3_End_QCTOT ~ L3_Int_QCTOT + L2_slope + L3_preference', data= data[(data.dropped == False) &  (data.L3_treatment == 1)]).fit() 
              
    else:
        data.dropped[i] = True
        
    if n_total >= n:
        break

0
9
13
14
21
24
41
42
43
48
51
57
60
70
73


/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


77
78
82
90
92
94
95
102
106
111
112
119
124
126
127
130
133
134
137
140
143
144
146
147
170
173
175
177
188
190
193
196
198
204
205
210
213
214
218
221
226
227
235
236
238
241
247
250
251
259
261
263
266
269
270
284
290
292
296
297
302
304
305
309
310
317
319
321
324
327
331
342
346
347
349
356
358
359
369
371
374
377
381
383
390
392
396
398
402
404
406
414
416
418
426
428
433
440
446
457
464
466
473
475
500
501
506
508
510
518
520
522
524
525
527
530
534
540
542
549
551
554
555
557
559
564
565
574
578
588
593
594
598
601
602
612
618
621
622
625
630
631
632
641
648
652
656
657
658
667
672
675
676
678
683
688
689
691
695
696
700
710
719
726
728
732
736
744
745
748
751
754
758
765
768
770
773
774
778
780
781
783
787
789
793
795
800
806
812
813
817
823
831
844
851
853
854
855
856
857
860
861
863
864
865
868
869
871
872
876
878
880
884
886
887
889
893
896
904
909
910
911
926
928
929
932
941
947
949
950
957
960
962
966
967
969
971
978
979
983
985
987
995
999
1002
1004
1009
1010
1015
1021
1

In [35]:
Reward = (data.L2_End_QCTOT[data.dropped == False] + data.L3_End_QCTOT[data.dropped == False])/2     
Reward.mean()

-7.825

In [36]:
temp = data[data.dropped == False]
temp.tail(50)

,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference,reward
1680,-18.0,-0.333333,1,-1,-8.0,-8.0,-0.714286,-1.0,-15.244056,0,...,False,0.032428,0.798402,1.000000,0.025891,38.0,18.0,0.333333,-1,-8.0
1689,-21.0,0.666667,-1,1,-7.0,-7.0,-1.000000,1.0,-13.449222,1,...,False,0.380973,0.226485,0.822357,0.070957,14.0,-21.0,0.666667,-1,-6.5
1693,-15.0,0.083333,-1,1,-15.0,-15.0,0.000000,-1.0,-27.500659,1,...,False,0.317043,0.563204,0.952010,0.169991,5.0,-15.0,0.083333,-1,-10.0
1701,-6.0,-0.833333,-1,1,-4.0,-4.0,-0.333333,1.0,-7.421307,0,...,False,0.235526,1.000000,1.000000,0.235526,4.0,-6.0,-0.833333,-1,-4.0
1702,-15.0,-0.333333,1,-1,-3.0,-3.0,-1.333333,-1.0,-7.241311,0,...,False,0.040547,1.000000,1.000000,0.040547,24.0,15.0,0.333333,-1,-3.0
1704,-12.0,0.000000,1,-1,-7.0,-7.0,-0.357143,-1.0,-13.075373,0,...,False,0.055820,0.519162,1.000000,0.028980,34.0,12.0,-0.000000,-1,-7.0
1706,-16.0,-0.142857,-1,1,-17.0,-17.0,0.111111,-1.0,-30.863010,1,...,False,0.349865,0.600386,0.951244,0.199813,5.0,-16.0,-0.142857,-1,-17.5
1714,-18.0,-0.333333,1,-1,-8.0,-8.0,-0.714286,-1.0,-15.244056,0,...,False,0.032428,0.798402,1.000000,0.025891,38.0,18.0,0.333333,-1,-8.0
1716,-19.0,0.333333,-1,1,-18.0,-18.0,-0.111111,-1.0,-32.872315,1,...,False,0.369086,0.634426,0.959535,0.224683,4.0,-19.0,0.333333,-1,-15.0
1718,-18.0,0.000000,-1,1,-15.0,-15.0,-0.214286,-1.0,-27.753788,1,...,False,0.375295,0.500201,0.954833,0.179244,5.0,-18.0,0.000000,-1,-14.5


In [37]:
(temp.L2_End_QCTOT/2 -temp.L2_Int_QCTOT + temp.L3_End_QCTOT/2).mean()

5.737

# offline optimal decision in paper

In [38]:
data['dropped'] = None

In [39]:
n_total = 0 #total number of points

#the first period beta's are 0
for i in data.index:
    
    if - data.L2_End_QCTOT[i] < 6:
        A_2 = 0
    elif -0.18 + 0.01* data.L3_Int_QCTOT[i] - 0.25* data.L2_slope[i] >0 :
        A_2 = 1
    else:
        A_2 = -1
        
    if -0.73 - 0.01* data.L2_Int_QCTOT[i] + 0.01*data.L1_slope[i] - 0.67*data.L2_preference[i]>0:
        A_1 = 1
    else:
        A_1 = -1
    
   
    # if disagrees, we drop the point; else, we update estimators
    if A_1 == data.L2_treatment[i] and A_2 == data.L3_treatment[i]:
        print(i)
        data.dropped[i] = False
        n_total = n_total + 1
      
    else:
        data.dropped[i] = True 
        
    if n_total >= n:
        break

/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/Cellar/jupyterlab/1.2.6/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


3
12
15
31
42
52
54
56
58
59
64
67
78
82
84
86
89
94
97
99
102
106
111
114
121
123
127
129
140
142
143
153
157
162
169
173
175
181
188
191
196
198
199
214
217
226
227
231
232
246
250
259
263
266
268
269
270
272
277
283
288
292
293
296
297
302
303
304
305
308
317
321
324
327
328
330
331
346
347
356
358
359
363
367
369
371
377
381
383
388
390
391
392
396
398
399
404
433
435
440
446
448
457
464
468
473
474
492
499
501
506
508
510
512
518
520
527
540
542
551
555
557
564
574
578
585
588
594
598
601
602
603
604
610
616
617
619
621
625
632
633
634
635
640
643
646
647
656
657
664
667
672
676
683
688
689
691
694
696
698
707
726
728
734
736
744
745
754
765
768
770
773
774
778
780
787
789
793
797
800
806
812
813
817
819
820
823
824
827
831
845
847
851
854
856
857
860
861
862
865
868
869
871
872
876
877
878
880
884
886
889
893
896
904
907
910
911
926
928
932
947
950
957
960
962
966
971
977
979
985
987
991
999
1002
1004
1005
1010
1020
1021
1022
1030
1032
1036
1041
1048
1049
1053
1055
1061
1063
1064

In [40]:
Reward = (data.L2_End_QCTOT[data.dropped == False] + data.L3_End_QCTOT[data.dropped == False])/2     
Reward.mean()

-7.34

In [41]:
temp = data[data.dropped == False]
temp.tail(50)

,L2_Int_QCTOT,L1_slope,L2_preference,L2_treatment,L2_End_QCTOT,L3_Int_QCTOT,L2_slope,L3_preference,L2_Pseudo,L3_Indicator,...,forced,p1,p0,p2,p,weight,A1_L2_Int_QCTOT,A1_L1_slope,A1_L2_preference,reward
2008,-8.0,-1.111111,-1,1,-8.0,-8.0,0.000000,-1.0,-15.273051,1,...,None,0.281659,0.297676,0.922739,0.077365,12.0,-8.0,-1.111111,-1,-8.0
2012,-12.0,0.000000,-1,1,-10.0,-10.0,-0.166667,-1.0,-18.963532,1,...,None,0.274225,0.409714,0.946093,0.106297,9.0,-12.0,0.000000,-1,-10.5
2013,-10.0,-0.416667,-1,1,-11.0,-11.0,0.047619,1.0,-19.198908,1,...,None,0.269327,0.442364,0.110019,0.013108,76.0,-10.0,-0.416667,-1,-8.5
2025,-9.0,-0.642857,-1,1,-9.0,-9.0,0.000000,-1.0,-17.019852,1,...,None,0.267988,0.359911,0.932432,0.089935,11.0,-9.0,-0.642857,-1,-10.5
2035,-20.0,0.142857,-1,1,-18.0,-18.0,-0.166667,-1.0,-32.937941,1,...,None,0.401432,0.598293,0.958880,0.230298,4.0,-20.0,0.142857,-1,-13.5
2037,-5.0,-2.166667,-1,-1,-4.0,-4.0,-0.083333,-1.0,-7.511527,0,...,None,0.698302,1.000000,1.000000,0.698302,1.0,5.0,2.166667,1,-4.0
2041,-12.0,0.000000,-1,1,-10.0,-10.0,-0.166667,-1.0,-18.963532,1,...,None,0.274225,0.409714,0.946093,0.106297,9.0,-12.0,0.000000,-1,-10.5
2043,-15.0,0.083333,-1,1,-15.0,-15.0,0.000000,-1.0,-27.500659,1,...,None,0.317043,0.563204,0.952010,0.169991,5.0,-15.0,0.083333,-1,-10.0
2067,-20.0,0.142857,-1,1,-18.0,-18.0,-0.166667,-1.0,-32.937941,1,...,None,0.401432,0.598293,0.958880,0.230298,4.0,-20.0,0.142857,-1,-13.5
2070,-11.0,-0.833333,1,-1,-1.0,-1.0,-0.714286,1.0,-3.016448,0,...,None,0.046952,1.000000,1.000000,0.046952,21.0,11.0,0.833333,-1,-1.0


In [42]:
(temp.L2_End_QCTOT/2 -temp.L2_Int_QCTOT + temp.L3_End_QCTOT/2).mean()

5.662